In [1]:
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup as bs

import my_passwords as ps
import my_functions as fn 

Cities

In [2]:
cities = [
    "Berlin",
    "Hamburg",
    "Munich",
    "Stuttgart"
]   

In [3]:
def get_send_cities_info():
    connection_string=fn.connect_to_sql(ps.my_password) # Connect to our database

    cities_df=fn.get_cities_info(cities) # Call the function get_cities_info() to generate the cities Dataframe
    fn.send_data_to_sql_without_truncate(cities_df.loc[:,["city","country_code"]],'cities',connection_string) # Send data about "city" and "country code" to the table "cities", we can not use truncate here since the city_id is a foreign key in other tables
    cities_from_sql = pd.read_sql("cities", con=connection_string) # Read the table "cities" from our database

    city_infos_df = cities_df.loc[:,["city","latitude","longitude","population","year_data_retrieved"]] # Extract the required columns from the cities Dataframe
    city_infos_df = city_infos_df.merge(cities_from_sql, on = "city", how="left") # Merge between the city_infos_df and cities_from_sql to get the corresponding city Id for each city infos
    city_infos_df = city_infos_df.drop(columns=["city","country_code"]) # Remove the columns "city" and "country_code" from the Dataframe
    city_infos_df = city_infos_df[["city_id","latitude","longitude","population","year_data_retrieved"]] 
    fn.send_data_to_sql_without_truncate(city_infos_df,'city_infos',connection_string) # Send data about "city_infos" to the table "city_infos"
    
    # These 2 lines are optional, they are used to print our SQL table 
    city_infos_from_sql = pd.read_sql("city_infos", con=connection_string) # Read the table "city_infos"
    print(city_infos_from_sql)

Weather

In [4]:
def get_send_weather_info():
    connection_string=fn.connect_to_sql(ps.my_password)
    cities_from_sql_df = pd.read_sql("cities", con=connection_string) # Read the table "cities" to get the list of cities
    weather_infos_df=fn.get_weather_info(cities_from_sql_df) # Call the function get_weather_info() to generate the weather Dataframe using the list of cities
    fn.send_data_to_sql_with_truncate(weather_infos_df,'weather_infos',connection_string) # Send data about "weather" to the table "weather_infos"
    # Optional code
    weather_infos_from_sql = pd.read_sql("weather_infos", con=connection_string) # Read the table "weather_infos"
    print(weather_infos_from_sql)

Airports

In [5]:
def get_send_airports_info():
    connection_string=fn.connect_to_sql(ps.my_password)
    cities_from_sql_df = pd.read_sql("cities", con=connection_string) # Read the table "cities" to get the list of cities
    airport_infos_df=fn.get_airpots_info(cities_from_sql_df) # Call the function get_airpots_info() to generate the airport Dataframe using the list of cities
    fn.send_data_to_sql_without_truncate(airport_infos_df.loc[:,["airport_icao","airport_name"]],'airports',connection_string) # Send data about "airport_icao" and "airport_name" to the table "airports"
    fn.send_data_to_sql_with_truncate(airport_infos_df.loc[:,["city_id","airport_icao"]],'cities_airports',connection_string) # Send data about "airport_icao" and "city_id" to the table "cities_airports"
    # Optional code
    airport_infos_from_sql = pd.read_sql("airports", con=connection_string) # Read the table "airports"
    print(airport_infos_from_sql)
    cities_airports_from_sql = pd.read_sql("cities_airports", con=connection_string) # Read the table "cities_airports"
    print(cities_airports_from_sql)

Flights

In [6]:
def get_send_flights_info():
    connection_string=fn.connect_to_sql(ps.my_password)
    airports_from_sql_df = pd.read_sql("airports", con=connection_string) # Read the table "airports" to get the list of airports
    flight_infos_df=fn.get_arrival_flights_info(airports_from_sql_df["airport_icao"]) # Call the function get_arrival_flights_info() to generate the flights Dataframe using the list of airports ICAO
    fn.send_data_to_sql_with_truncate(flight_infos_df,"flights",connection_string) # Send data about "flights" to the table "flights"
    # Optional code
    flight_infos_from_sql = pd.read_sql("flights", con=connection_string)
    print(flight_infos_from_sql)

In [7]:
# This function call all the previous functions to create all the tables of our database
def create_all_tables():
    get_send_cities_info()
    get_send_weather_info()
    get_send_airports_info()
    get_send_flights_info()

In [8]:
create_all_tables()

   city_id    latitude   longitude  population  year_data_retrieved
0        1  52°31′12″N  13°24′18″E     3576873                 2024
1        2     53°33′N     10°00′E     1945532                 2022
2        3  48°08′15″N  11°34′30″E     1512491                 2022
3        4  48°46′39″N  09°10′48″E      626275                 2021
     weather_id  city_id        weather_time data_collected_time  temperature  \
0             1        1 2024-03-18 00:00:00 2024-03-17 22:49:23          3.0   
1             2        1 2024-03-18 03:00:00 2024-03-17 22:49:23          2.0   
2             3        1 2024-03-18 06:00:00 2024-03-17 22:49:23          1.0   
3             4        1 2024-03-18 09:00:00 2024-03-17 22:49:23          4.0   
4             5        1 2024-03-18 12:00:00 2024-03-17 22:49:23          7.0   
..          ...      ...                 ...                 ...          ...   
155         156        4 2024-03-22 09:00:00 2024-03-17 22:49:24         15.0   
156         